In [1]:
from shapely.geometry import Point
import geopandas as gpd
import requests
import pandas as pd
import re
import json
import math
import os       

In [2]:
df_bicimad = pd.read_csv('../data/bicimad_stations.csv',delimiter ='\t')
df_bicimad.head()

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"


In [3]:
df_bicimad['geometry.coordinates'][0]

'[-3.7018341, 40.4172137]'

In [4]:
df_bicimad['longitude'] = df_bicimad['geometry.coordinates'].apply(lambda x: float(x.split(',')[0].replace('[', ''))) 
df_bicimad['latitude'] = df_bicimad['geometry.coordinates'].apply(lambda x: float(x.split(',')[1].replace(']', '')))
df_bicimad.head()

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.701834,40.417214
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]",-3.701603,40.417313
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]",-3.705842,40.420589
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]",-3.706917,40.430294
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]",-3.702587,40.428552


In [5]:
df_bicipark = pd.read_csv('../data/bicipark_stations.csv', delimiter = ';')
df_bicipark.head()


,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]"
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]"
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]"
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]"
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]"


In [6]:
df_bicipark['longitude'] = df_bicipark['geometry.coordinates'].apply(lambda x: float(x.split(',')[0].replace('[', ''))) 
df_bicipark['latitude'] = df_bicipark['geometry.coordinates'].apply(lambda x: float(x.split(',')[1].replace(']', '')))
df_bicipark.head()

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,longitude,latitude
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]",-3.663068,40.484684
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]",-3.693897,40.452973
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]",-3.690786,40.418146
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]",-3.692879,40.430361
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]",-3.692245,40.420354


In [7]:
url = "https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json"
response = requests.get(url)
print(response.status_code)

200


In [8]:
json_data = response.json()
data = json_data['@graph']
print(data)

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/7933-consulado-argentina.json', '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/OrganismosInternacionales/EmbajadasConsulados', 'id': '7933', 'title': 'Consulado de Argentina', 'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=654df3216571c010VgnVCM1000000b205a0aRCRD', 'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Chamberi'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Chamberi/Barrio/Almagro'}, 'locality': 'MADRID', 'postal-code': '28010', 'street-address': 'CALLE FERNANDO EL SANTO 15 PLANTA BAJA'}, 'location': {'latitude': 40.42821624776458, 'longitude': -3.6926659313220473}, 'organization': {'organization-desc': '\xa0\xa0Metro: Alonso Martínez (líneas 4, 5 y 10).  Bus: 7, 21, C03, 37, 3, 14, 150, 27, 45, 5.  Bicimad: E

In [9]:
df_embajadas = pd.DataFrame.from_dict(data)

In [10]:
df_embajadas.iloc[14]['location']

nan

In [11]:
df_embajadas = df_embajadas[df_embajadas['location'].notna()]
df_embajadas = df_embajadas.reset_index(drop=True)
df_embajadas['longitude'] = df_embajadas['location'].apply(lambda x: x['longitude']) 
df_embajadas['latitude'] = df_embajadas['location'].apply(lambda x: x['latitude'])
df_embajadas= df_embajadas.drop('location', axis=1)
df_embajadas.head()

,@id,@type,id,title,relation,address,organization,longitude,latitude
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Alonso Martíne...,-3.692666,40.428216
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Concha Espina ...,-3.680978,40.453376
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Cruz del Rayo ...,-3.679665,40.442447
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.688868,40.425716
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.689004,40.428626


In [12]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish).values[0]

In [13]:
def get_min_distance(lat_origin, long_origin, bici_points):   # bici_points = [[lat, long], [lat, long], [lat, long]] 
	closest_station = None
	min_distance = 1000000.0
	for station in bici_points:
		distance = float(distance_meters(lat_origin, long_origin, station[0], station[1])) # una distancia entre una embajada cualquiera y una estacion cualquiera -2km
		if distance < min_distance:
			min_distance = distance 
			closest_station = station
	return closest_station # devuelve la lat y long de la bicis

In [14]:
df_embajadas.columns

Index(['@id', '@type', 'id', 'title', 'relation', 'address', 'organization',
       'longitude', 'latitude'],
      dtype='object')

In [15]:
bicipark_points = df_bicipark[['latitude', 'longitude']].values.tolist()

In [16]:
bicipark_points

[[40.4846838, -3.6630679],
 [40.452973, -3.693897],
 [40.418146, -3.690786],
 [40.430361, -3.692879],
 [40.420354, -3.692245],
 [40.472419, -3.678921],
 [40.430315, -3.679172],
 [40.425608, -3.693698],
 [40.4959425, -3.7249142],
 [40.383491516, -3.78369522],
 [40.417807676, -3.6994506]]

In [17]:
df_embajadas['closest_bicipark_point'] = df_embajadas.apply(lambda embajada: get_min_distance(embajada['latitude'], embajada['longitude'], bicipark_points), axis=1) #closest_station

In [18]:
df_embajadas.columns

Index(['@id', '@type', 'id', 'title', 'relation', 'address', 'organization',
       'longitude', 'latitude', 'closest_bicipark_point'],
      dtype='object')

In [19]:
bicimad_points = df_bicimad[['latitude', 'longitude']].values.tolist()

In [20]:
bicimad_points

[[40.4172137, -3.7018341],
 [40.41731271011562, -3.701602938060457],
 [40.4205886, -3.7058415],
 [40.4302937, -3.7069171],
 [40.4285524, -3.7025875],
 [40.42852, -3.70205],
 [40.424148, -3.698447],
 [40.4251906, -3.6977715],
 [40.4278682, -3.6954403],
 [40.4156057, -3.7095084],
 [40.42533, -3.69214],
 [40.4269483, -3.7035918],
 [40.4284246, -3.7061931],
 [40.4273264, -3.7104417],
 [40.4260957, -3.713479],
 [40.4262383, -3.7074453],
 [40.4230721, -3.7075065],
 [40.4232649, -3.7038312],
 [40.4207773, -3.6996502],
 [40.4218616, -3.6954983],
 [40.4192342, -3.6954615],
 [40.4200783, -3.7065376],
 [40.4202834, -3.7081246],
 [40.417908, -3.710692],
 [40.4173703, -3.7057791],
 [40.4172781, -3.7063837],
 [40.423855, -3.694475],
 [40.4132798, -3.6956178],
 [40.4181518, -3.6984368],
 [40.4162619, -3.6957355],
 [40.4192095, -3.711504],
 [40.4163638, -3.7068969],
 [40.4150099, -3.7061032],
 [40.4165039, -3.7030833],
 [40.4146755, -3.7036825],
 [40.4141931, -3.7103285],
 [40.4168, -3.71183],
 [40.41

In [21]:
df_embajadas.columns

Index(['@id', '@type', 'id', 'title', 'relation', 'address', 'organization',
       'longitude', 'latitude', 'closest_bicipark_point'],
      dtype='object')

In [22]:
df_embajadas['closest_bicimad_point'] = df_embajadas.apply(lambda embajada: get_min_distance(embajada['latitude'], embajada['longitude'], bicimad_points), axis=1) #closest_station

In [23]:
df_embajadas.head()

,@id,@type,id,title,relation,address,organization,longitude,latitude,closest_bicipark_point,closest_bicimad_point
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Alonso Martíne...,-3.692666,40.428216,"[40.430361, -3.692879]","[40.4278682, -3.6954403]"
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Concha Espina ...,-3.680978,40.453376,"[40.452973, -3.693897]","[40.4510188, -3.6817962]"
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Cruz del Rayo ...,-3.679665,40.442447,"[40.430315, -3.679172]","[40.4442258, -3.6787169]"
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.688868,40.425716,"[40.425608, -3.693698]","[40.4257046, -3.6893698]"
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.689004,40.428626,"[40.430361, -3.692879]","[40.42677, -3.68958]"


In [24]:
def get_bicipark(coords_station): #[lat, long]  --> estacion tal
    df_station = df_bicipark[(df_bicipark['latitude'] == coords_station[0]) & (df_bicipark['longitude'] == coords_station[1])] 
    station = df_station.iloc[0]
    return station
df_embajadas['Bicipark_station_name'] = df_embajadas['closest_bicipark_point'].apply(lambda x: get_bicipark(x)['stationName'])


In [25]:
def get_bicimad(coords_station): #[lat, long]  --> estacion tal
    df_station = df_bicimad[(df_bicimad['latitude'] == coords_station[0]) & (df_bicimad['longitude'] == coords_station[1])] 
    station = df_station.iloc[0]
    return station
df_embajadas['Bicimad_station_name'] = df_embajadas['closest_bicimad_point'].apply(lambda x: get_bicimad(x)['name'])


In [26]:
df_embajadas['Bicipark_station_address'] = df_embajadas['closest_bicipark_point'].apply(lambda x: get_bicipark(x)['address'])


In [27]:
df_embajadas['Bicimad_station_address'] = df_embajadas['closest_bicimad_point'].apply(lambda x: get_bicimad(x)['address'])


In [28]:
df_embajadas.head()

,@id,@type,id,title,relation,address,organization,longitude,latitude,closest_bicipark_point,closest_bicimad_point,Bicipark_station_name,Bicimad_station_name,Bicipark_station_address,Bicimad_station_address
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Alonso Martíne...,-3.692666,40.428216,"[40.430361, -3.692879]","[40.4278682, -3.6954403]",Bicipark Almagro,8 - Alonso Martínez,"Calle Almagro, 11",Plaza de Alonso Martínez nº 5
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Concha Espina ...,-3.680978,40.453376,"[40.452973, -3.693897]","[40.4510188, -3.6817962]",Bicipark Orense,148 - Serrano 210,"Avenida General Perón, 27",Calle Serrano nº 210
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Cruz del Rayo ...,-3.679665,40.442447,"[40.430315, -3.679172]","[40.4442258, -3.6787169]",Bicipark Salamanca,146 - María Francisca 1,Pl. del Marqués de Salamanca,Calle María Francisca nº 1
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.688868,40.425716,"[40.425608, -3.693698]","[40.4257046, -3.6893698]",Bicipark Villa de Paris,93 - Plaza de Colón,"Calle de Orellana, 23, 21",Calle Goya nº 1
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,{'organization-desc': ' Metro: Serrano (línea...,-3.689004,40.428626,"[40.430361, -3.692879]","[40.42677, -3.68958]",Bicipark Almagro,104 - Castellana,"Calle Almagro, 11",Paseo de la Castellana nº 4


In [29]:
df_embajadas.iloc[0]['address']

{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Chamberi'},
 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Chamberi/Barrio/Almagro'},
 'locality': 'MADRID',
 'postal-code': '28010',
 'street-address': 'CALLE FERNANDO EL SANTO 15 PLANTA BAJA'}

In [30]:
df_embajadas['address'] = df_embajadas['address'].apply(lambda embajada: embajada['street-address'])

In [31]:
df_embajadas.head()

,@id,@type,id,title,relation,address,organization,longitude,latitude,closest_bicipark_point,closest_bicimad_point,Bicipark_station_name,Bicimad_station_name,Bicipark_station_address,Bicimad_station_address
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,{'organization-desc': ' Metro: Alonso Martíne...,-3.692666,40.428216,"[40.430361, -3.692879]","[40.4278682, -3.6954403]",Bicipark Almagro,8 - Alonso Martínez,"Calle Almagro, 11",Plaza de Alonso Martínez nº 5
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CALLE TALAVERA 9,{'organization-desc': ' Metro: Concha Espina ...,-3.680978,40.453376,"[40.452973, -3.693897]","[40.4510188, -3.6817962]",Bicipark Orense,148 - Serrano 210,"Avenida General Perón, 27",Calle Serrano nº 210
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CALLE AVIADOR LINDBERGH 3,{'organization-desc': ' Metro: Cruz del Rayo ...,-3.679665,40.442447,"[40.430315, -3.679172]","[40.4442258, -3.6787169]",Bicipark Salamanca,146 - María Francisca 1,Pl. del Marqués de Salamanca,Calle María Francisca nº 1
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CALLE GOYA 5 y 7 pasaje,{'organization-desc': ' Metro: Serrano (línea...,-3.688868,40.425716,"[40.425608, -3.693698]","[40.4257046, -3.6893698]",Bicipark Villa de Paris,93 - Plaza de Colón,"Calle de Orellana, 23, 21",Calle Goya nº 1
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,PASEO CASTELLANA 18 PLANTA 6 Izquierda,{'organization-desc': ' Metro: Serrano (línea...,-3.689004,40.428626,"[40.430361, -3.692879]","[40.42677, -3.68958]",Bicipark Almagro,104 - Castellana,"Calle Almagro, 11",Paseo de la Castellana nº 4


In [32]:
def get_type_of_place(place_name):
    place_name = place_name.lower() # name en minusculas
    if 'consulado' in place_name:
        return 'Consulado'
    else:
        return 'Embajada'

In [33]:
df_embajadas['Type of Place'] = df_embajadas['title'].apply(lambda embajada_name: get_type_of_place(embajada_name))
df_embajadas = df_embajadas[df_embajadas.columns[df_embajadas.columns.isin(['title', 'Type of Place', 'address', 'Bicipark_station_name', 'Bicipark_station_address', 'Bicimad_station_name', 'Bicimad_station_address' ])]] # botando lo que no necesito
df_embajadas = df_embajadas[['title', 'Type of Place', 'address', 'Bicipark_station_name', 'Bicipark_station_address', 'Bicimad_station_name', 'Bicimad_station_address' ]] # reordenar columnas
df_embajadas = df_embajadas.rename(columns={'title': 'Place of interest', 'address': 'Place address'}) # renaming las columnas

In [34]:
df_embajadas.head()

,Place of interest,Type of Place,Place address,Bicipark_station_name,Bicipark_station_address,Bicimad_station_name,Bicimad_station_address
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,Bicipark Almagro,"Calle Almagro, 11",8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
1,Consulado de Belice,Consulado,CALLE TALAVERA 9,Bicipark Orense,"Avenida General Perón, 27",148 - Serrano 210,Calle Serrano nº 210
2,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,Bicipark Salamanca,Pl. del Marqués de Salamanca,146 - María Francisca 1,Calle María Francisca nº 1
3,Consulado de Brasil,Consulado,CALLE GOYA 5 y 7 pasaje,Bicipark Villa de Paris,"Calle de Orellana, 23, 21",93 - Plaza de Colón,Calle Goya nº 1
4,Consulado de Bélgica,Consulado,PASEO CASTELLANA 18 PLANTA 6 Izquierda,Bicipark Almagro,"Calle Almagro, 11",104 - Castellana,Paseo de la Castellana nº 4
